<a href="https://colab.research.google.com/github/varun19424/Next_Word_Predictor_Using_LSTM/blob/main/Next_Word_Predictor_Using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [3]:
with open('sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [5]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [6]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [7]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [8]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [9]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2208800 (8.43 MB)
Trainable params: 2208800 (8.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [10]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
3010/3010 [==============================] - 35s 11ms/step - loss: 6.2482 - accuracy: 0.0749
Epoch 2/100
3010/3010 [==============================] - 22s 7ms/step - loss: 5.5210 - accuracy: 0.1232
Epoch 3/100
3010/3010 [==============================] - 20s 7ms/step - loss: 5.1260 - accuracy: 0.1486
Epoch 4/100
3010/3010 [==============================] - 21s 7ms/step - loss: 4.7920 - accuracy: 0.1665
Epoch 5/100
3010/3010 [==============================] - 20s 7ms/step - loss: 4.4846 - accuracy: 0.1833
Epoch 6/100
3010/3010 [==============================] - 20s 7ms/step - loss: 4.1925 - accuracy: 0.2031
Epoch 7/100
3010/3010 [==============================] - 20s 7ms/step - loss: 3.9162 - accuracy: 0.2289
Epoch 8/100
3010/3010 [==============================] - 20s 6ms/step - loss: 3.6556 - accuracy: 0.2596
Epoch 9/100
3010/3010 [==============================] - 19s 6ms/step - loss: 3.4100 - accuracy: 0.2934
Epoch 10/100
3010/3010 [==============================] - 20s 7

In [13]:
seed_text = "To Sherlock Holmes she is"
next_words = 10

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 18ms/step
To Sherlock Holmes she is always the woman i have seldom heard my father i
